In [ ]:
import csv
import os
from dotenv import load_dotenv
import pandas as pd

load_dotenv()

cnous_filepath = os.environ['CNOUS_PATHFILE']
msa_filepath = os.environ['MSA_PATHFILE']
cnaf_filepath = os.environ['CNAF_PATHFILE']

In [ ]:
cnous_df = pd.read_csv(cnous_filepath, nrows=5000, encoding='iso-8859-1', on_bad_lines='skip', sep=',')

In [ ]:
msa_df = pd.read_csv(msa_filepath, nrows=5000, encoding='utf-8', on_bad_lines='skip', sep=';', quoting=csv.QUOTE_NONE)

In [ ]:
cnaf_df = pd.read_csv(cnaf_filepath, nrows=5000, encoding='iso-8859-1', on_bad_lines='skip', sep=';', quoting=csv.QUOTE_NONE)

## Mapping des données MSA

In [ ]:
msa_column_mapping = {
    'numero_allocataire': 'allocataire-matricule',
    'organisme': 'allocataire-code_organisme',
    'qualite_allocataire': 'allocataire-qualite',
    'nom_allocataire': 'allocataire-nom',
    'prenom_allocataire': 'allocataire-prenom',
    'commune_de_naissance': 'allocataire-commune_naissance',
    'code_insee_commune_alloc':'allocataire-code_insee_commune_naissance',
    'pays_de_naissance': 'allocataire-pays_naissance',
    'code_iso_pays': 'allocataire-code_iso_pays_naissance',
    'date_de_naissance_alloc': 'allocataire-date_naissance',
    'adresse_de_messagerie': 'allocataire-courriel',
    'numero_tel_portable': 'allocataire-telephone',
    'complement_adresse' :'adresse-allocataire-cplt_adresse',
    'nom_commune': 'adresse-allocataire-commune',
    'code_postal': 'adresse-allocataire-code_postal',
    'code_insee_commune': 'adresse-allocataire-code_insee',
    'nom_beneficiaire': 'nom',
    'prenom_beneficiaire': 'prenom',
    'genre_beneficiaire': 'sexe',
    'date_de_naissance_beneficiaire': 'date_naissance'
}

df_psp_mapped_msa = msa_df.copy()
df_psp_mapped_msa = df_psp_mapped_msa.rename(columns=msa_column_mapping) 

# situation
df_psp_mapped_msa['situation'] = 'Jeune'
condition_1 = df_psp_mapped_msa['allocataire-nom'] == df_psp_mapped_msa['nom'] 
condition_2 = df_psp_mapped_msa['allocataire-prenom'] == df_psp_mapped_msa['prenom']
df_psp_mapped_msa.loc[condition_1 & condition_2, 'situation'] =  'AAH'

# organisme
df_psp_mapped_msa['organisme'] = 'MSA'

# nom adresse postale de l'allocataire
df_psp_mapped_msa['adresse-allocataire-nom_adresse_postale'] = msa_df['qualite_destinataire'] + ' ' + msa_df['nom_destinataire'] + ' ' + msa_df['prenom_destinataire']

# voie de l'adresse postale de l'allocataire
df_psp_mapped_msa['adresse-allocataire-voie'] = msa_df['numero_voie'] + ' ' + msa_df['complement_numero_voie'] + ' ' + msa_df['type_voie'] + msa_df['voie']

# sexe
df_psp_mapped_msa['sexe'] = df_psp_mapped_msa['sexe'].replace('1', 'M')
df_psp_mapped_msa['sexe'] = df_psp_mapped_msa['sexe'].replace('2', 'F')

# qualite allocataire
df_psp_mapped_msa['allocataire-qualite'] = df_psp_mapped_msa['allocataire-qualite'].replace('MME', 'Mme')
df_psp_mapped_msa['allocataire-qualite'] = df_psp_mapped_msa['allocataire-qualite'].replace('MR', 'M')

# code commune de naissance de l'allocataire
df_psp_mapped_msa['allocataire-code_insee_commune_naissance'] = df_psp_mapped_msa['allocataire-code_insee_commune_naissance'].replace('0', None)

# date de naissance allocataire
df_psp_mapped_msa['allocataire-date_naissance'] = df_psp_mapped_msa['allocataire-date_naissance'].replace(0, None)
df_psp_mapped_msa['allocataire-date_naissance'] = pd.to_datetime(df_psp_mapped_msa['allocataire-date_naissance'], format='%Y%m%d')
df_psp_mapped_msa['allocataire-date_naissance'] = df_psp_mapped_msa['allocataire-date_naissance'].dt.strftime('%d/%m/%Y')

# date de naissance bénéficiaire
df_psp_mapped_msa['date_naissance'] = pd.to_datetime(df_psp_mapped_msa['date_naissance'], format='%Y%m%d')
df_psp_mapped_msa['date_naissance'] = df_psp_mapped_msa['date_naissance'].dt.strftime('%d/%m/%Y')

## Mapping des données CNAF

In [ ]:
cnaf_column_mapping = {
    'numorg': 'allocataire-code_organisme',
    'matricul': 'allocataire-matricule',
    'qualite': 'allocataire-qualite',
    'nom': 'allocataire-nom',
    'prenom': 'allocataire-prenom',
    'dt_naissance': 'allocataire-date_naissance',
    'adresse': 'adresse_allocataire-voie',
    'code_postal': 'adresse_allocataire-code_postal',
    'commune': 'adresse_allocataire-commune',
    'code_insee': 'adresse_allocataire-code_insee',
    'mail': 'allocataire-courriel',
    'tel': 'allocataire-telephone',
    'dt_naissance_eligible': 'date_naissance',
    'nom_eligible': 'nom',
    'prenom_eligible': 'prenom',
    'sexe_eligible': 'genre',
    'prestation': 'situation organisme'  # Note: Multiple columns mapped to one field
}

df_psp_mapped_cnaf = cnaf_df.copy()
df_psp_mapped_cnaf = df_psp_mapped_cnaf.rename(columns=cnaf_column_mapping) 

# qualité allocataire
df_psp_mapped_cnaf['allocataire-qualite'] = df_psp_mapped_cnaf['allocataire-qualite'].replace('Madame', 'Mme')
df_psp_mapped_cnaf['allocataire-qualite'] = df_psp_mapped_cnaf['allocataire-qualite'].replace('Monsieur', 'M')

# voie de l'adresse postale de l'allocataire
df_psp_mapped_cnaf['adresse_allocataire-cplt'] = cnaf_df['compl_adresse1'] + ' ' + cnaf_df['compl_adresse2'] + ' ' + cnaf_df['compl_adresse3']

# organisme
df_psp_mapped_cnaf['organisme'] = 'CAF'

# sexe
df_psp_mapped_cnaf['genre'] = df_psp_mapped_cnaf['genre'].replace('Masculin', 'M')
df_psp_mapped_cnaf['genre'] = df_psp_mapped_cnaf['genre'].replace('Féminin', 'F')

# situation
df_psp_mapped_cnaf['situation'] = 'Jeune'
condition_1 = df_psp_mapped_cnaf['allocataire-nom'] == df_psp_mapped_cnaf['nom'] 
condition_2 = df_psp_mapped_cnaf['allocataire-prenom'] == df_psp_mapped_cnaf['prenom']
df_psp_mapped_cnaf.loc[condition_1 & condition_2, 'situation'] =  'AAH'



## Mapping des données CNOUS

In [ ]:
# TODO
cnous_column_mapping = {
    'ine': 'allocataire.matricule',
    'nom': 'nom allocataire.nom',
    'prenom': 'prenom allocataire.prenom',
    'dateNaissance': 'date_naissance',
    'civiliteLibelleCourt': 'genre allocataire.qualite',
    'lieuNaissCodeCommuneInsee': 'allocataire.code_insee_commune_naissance',
    'lieuNaissLibelleCommune': 'allocataire.commune_naissance',
    'lieuNaissCodePays': 'allocataire.code_iso_pays_naissance',
    'lieuNaissLibellePays': 'allocataire.pays_naissance',
    'mail': 'allocataire.courriel',
    'adresseVoie': 'adresse_allocataire.voie',
    'adresseComplement1': 'adresse_allocataire.cplt_adresse',
    'adresseComplement2': '',
    'adresseLocalite': 'adresse_allocataire.commune',
    'adresseCodePostal': 'adresse_allocataire.code_postal',
    'adresseCodePays': '-',
    'adresseCodeLocalite': 'adresse_allocataire.code_insee',
    '-': 'situation'
}


df_psp_mapped_cnous = cnous_df.copy()
df_psp_mapped_cnous = df_psp_mapped_cnous.rename(columns=cnous_column_mapping) 

# organisme
df_psp_mapped_cnous['organisme'] = 'cnous'


# Merge dans un seul dataframe cible pour BDD Postgresql

In [ ]:
df_all = pd.concat([df_psp_mapped_cnaf, df_psp_mapped_cnous, df_psp_mapped_msa], axis=0, ignore_index=True)

In [ ]:
df_all['date_naissance'] = pd.to_datetime(df_all['date_naissance'], format='%d/%m/%Y')


# Application des critères d'éligibilité

In [ ]:
from datetime import datetime

# ARS

start_date = datetime(2006, 9, 16).date()
end_date = datetime(2018, 12, 31).date()

ars_situation_mask = ((df_all['organisme'] == 'CAF') | (df_all['organisme'] == 'MSA')) & (df_all['date_naissance'].dt.date >= start_date) & (df_all['date_naissance'].dt.date <= end_date) & (df_all['situation'] == 'Jeune')
df_all_filtered_ars = df_all[ars_situation_mask]

In [ ]:
# AEEH

start_date = datetime(2004, 6, 1).date()
end_date = datetime(2018, 12, 31).date()

aeeh_situation_mask = ((df_all['organisme'] == 'CAF') | (df_all['organisme'] == 'MSA')) & (df_all['date_naissance'].dt.date >= start_date) & (df_all['date_naissance'].dt.date <= end_date) & (df_all['situation'] == 'Jeune')
df_all_filtered_aeeh = df_all[aeeh_situation_mask]

In [ ]:
# AAH

start_date = datetime(1993, 9, 16).date()
end_date = datetime(2008, 12, 31).date()

aah_situation_mask = ((df_all['organisme'] == 'CAF') | (df_all['organisme'] == 'MSA')) & (df_all['date_naissance'].dt.date >= start_date) & (df_all['date_naissance'].dt.date <= end_date) & (df_all['situation'] == 'AAH')
df_all_filtered_aah = df_all[aah_situation_mask]

In [ ]:
# CNOUS

from datetime import timedelta

end_date = pd.to_datetime('today').date()
start_date = end_date - timedelta(days=28*365)

cnous_situation_mask = (df_all['organisme'] == 'cnous') & (df_all['date_naissance'].dt.date >= start_date) & (df_all['date_naissance'].dt.date <= end_date)
df_all_filtered_cnous = df_all[cnous_situation_mask]